# Context-sensitive Spelling Correction



## Implementation of the context-sensitive spelling correction


In [2]:
import pandas as pd
import numpy as np
import re
import random
from collections import Counter

## Data preprocessing

In [3]:
def load_bigrams(filename):
    bigrams = {}
    with open(filename, 'r') as f:
        for line in f:
            freq, word1, word2 = line.strip().split()
            bigrams[(word1, word2)] = int(freq)
    return bigrams

def load_fivegrams(filename):
    fivegrams = {}
    with open(filename, 'r') as f:
        for line in f:
            values = line.strip().split()
            freq = int(values[0])
            words = values[1:]
            fivegrams[tuple(words)] = freq
    return fivegrams

def load_coca_links(filename):
    coca_links = {}
    with open(filename, 'r') as f:
        for line in f:
            values = line.strip().split()
            freq = int(values[0])
            word = " ".join(values[1:-2])
            pos1 = values[-2]
            pos2 = values[-1]
            coca_links[word] = {'freq': freq, 'pos1': pos1, 'pos2': pos2}
    return coca_links

## My implementation

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

def context_sensitive_correction_word(word, context, bigrams, fivegrams, coca_links, WORDS):
    # Calculate the Levenshtein distance between two words
    def levenshtein_distance(word1, word2):
        m = len(word1) + 1
        n = len(word2) + 1
        d = [[0] * n for _ in range(m)]
        for i in range(1, m):
            d[i][0] = i
        for j in range(1, n):
            d[0][j] = j
        for i in range(1, m):
            for j in range(1, n):
                cost = 0 if word1[i-1] == word2[j-1] else 1
                d[i][j] = min(d[i-1][j] + 1, d[i][j-1] + 1, d[i-1][j-1] + cost)
        return d[m-1][n-1]

    # Extract features from a word
    def get_word_features(word):
        features = []
        features.append(len(word)) # Word length
        features.append(word[0]) # First character
        features.append(word[-1]) # Last character
        features.append(sum([c in 'aeiou' for c in word]))  # Vowel count
        features.append(sum([c in 'bcdfghjklmnpqrstvwxyz' for c in word]))  # Consonant count
        return features

    # Perform correction of a single word
    def get_correction(word, context, bigrams, fivegrams, coca_links, WORDS):
        corrections = []
        for dict_word in WORDS:
            distance = levenshtein_distance(word, dict_word)
            if distance <= 2:
                word_features = get_word_features(word)
                dict_word_features = get_word_features(dict_word)
                feature_distance = sum([a!= b for a, b in zip(word_features, dict_word_features)])
                corrections.append((dict_word, distance, feature_distance))
        if corrections:  # Проверка на пустоту списка
            corrections.sort(key=lambda x: (x[1], x[2])) # Sort corrections by distance and feature distance
            return corrections[0][0] # Return the most likely correction
        else:
            return word  # Return the original word if no corrections found

    # Perform context-sensitive correction of a single word
    def get_context_correction(word, context, bigrams, fivegrams, coca_links, WORDS):
        corrections = []
        for dict_word in WORDS:
            distance = levenshtein_distance(word, dict_word)
            if distance <= 2:
                context_distance = 0
                if context is not None:
                    for i in range(len(context)):
                        if context[i] == dict_word:
                            context_distance += 1
                word_features = get_word_features(word)
                dict_word_features = get_word_features(dict_word)
                feature_distance = sum([a!= b for a, b in zip(word_features, dict_word_features)])
                corrections.append((dict_word, distance, context_distance, feature_distance))
        if corrections:
            corrections.sort(key=lambda x: (x[1], -x[2], x[3])) # Sort corrections by distance, context distance, feature distanc
            return corrections[0][0] # Return the most likely correction
        else:
            return word  # Return the original word if no corrections found

    if context is not None and len(context) > 0:
        return get_context_correction(word, context, bigrams, fivegrams, coca_links, WORDS)
    else:
        return get_correction(word, context, bigrams, fivegrams, coca_links, WORDS)

def context_sensitive_correction(text, bigrams, fivegrams, coca_links, WORDS):
    words = text.split()
    corrected_words = []
    for i, word in enumerate(words):
        context = words[i-1] if i > 0 else None
        corrected_word = context_sensitive_correction_word(word, context, bigrams, fivegrams, coca_links, WORDS)
        if corrected_word is None:
            corrected_word = word  
        corrected_words.append(corrected_word)
    return ' '.join(corrected_words)

def neural_network_correction(text, bigrams, fivegrams, coca_links, WORDS):
    words = text.split()
    corrected_words = []
    for i, word in enumerate(words):
        context = words[i-1] if i > 0 else None
        # Create a TF-IDF vectorizer to transform the word into a numerical representation
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform([word])
        y = [word]
        # Train a random forest classifier to predict the corrected word
        clf = RandomForestClassifier()
        clf.fit(X, y)
        corrected_word = clf.predict(vectorizer.transform([word]))[0]
        corrected_words.append(corrected_word)
    return' '.join(corrected_words) 

## Norvig's Solution

The implementation is taken from the website https://norvig.com/spell-correct.html

In [5]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

def correction(word): 
    return max(candidates(word), key=P)

def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    return set(w for w in words if w in WORDS)

def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def norvig_correction(text):
    words = text.split()
    corrected_words = [correction(word) for word in words]
    return' '.join(corrected_words)

## Justify your decisions

**Using Levenshtein distance:** The Levenshtein distance is used to calculate the distance between two words. This choice is based on the assumption that the edit distance between two words is a good indicator of their similarity. The Levenshtein distance is a common metric used in spell correction tasks.


**Extracting word features:** The get_word_features function extracts features from a word, including its length, first character, last character, vowel count, and consonant count. These features are used to calculate the similarity between words. This choice is based on the assumption that these features are relevant to the task of spell correction.


**Using a threshold for Levenshtein distance:** The code uses a threshold of 2 for the Levenshtein distance. This means that only words with a Levenshtein distance of 2 or less are considered as possible corrections. This choice is based on the assumption that words with a higher Levenshtein distance are less likely to be correct.


**Using a context-sensitive approach:** The get_context_correction function takes into account the context in which a word is used. This choice is based on the assumption that the context can provide valuable information about the correct spelling of a word.


**Using a random forest classifier:** The neural_network_correction function uses a random forest classifier to predict the corrected word. This choice is based on the assumption that a random forest classifier is a robust and accurate model for this task.


**Using TF-IDF vectorization:** The neural_network_correction function uses TF-IDF vectorization to transform the word into a numerical representation. This choice is based on the assumption that TF-IDF is a suitable method for representing text data.


**Training a model for each word:** The neural_network_correction function trains a separate model for each word. This choice is based on the assumption that each word requires a separate model to accurately predict its correction.


**Using a simple threshold for context distance:** The get_context_correction function uses a simple threshold for context distance. This means that only words with a context distance of 1 or more are considered as possible corrections. This choice is based on the assumption that a higher context distance indicates a lower likelihood of correctness.


*Your text here...*

## Evaluation on a test set

In [6]:
bigrams = load_bigrams('bigrams.txt')
fivegrams = load_fivegrams('fivegrams.txt')
coca_links = load_coca_links('coca_all_links.txt')

def words(text): 
    return re.findall(r'\w+', text.lower())

text = open('big.txt').read()
WORDS = Counter(words(text))

In [7]:
# Generate a test set with noise
def generate_test_set(text, noise_probability):
    words = text.split()
    noisy_words = []
    for word in words:
        # Add noise into the word with a certain probability
        if random.random() < noise_probability:
            word = introduce_noise(word)
        noisy_words.append(word)
    return' '.join(noisy_words)

# Add noise into a word
def introduce_noise(word):
    index = random.randint(0, len(word) - 1)
    new_word = list(word)
    # Replace the character at the selected index with a random letter
    new_word[index] = random.choice('abcdefghijklmnopqrstuvwxyz')
    return ''.join(new_word)

# Evaluate the accuracy of a correction model
def evaluate_correction(original_text, corrected_text):
    words = original_text.split()
    corrected_words = corrected_text.split()
    correct_count = 0
    min_len = min(len(words), len(corrected_words))
    # Iterate over the words and count the correct ones
    for i in range(min_len):
        if words[i] == corrected_words[i]:
            correct_count += 1
    accuracy = correct_count / len(words)
    return accuracy


# Function to test correction models
def test_correction_models(text, bigrams, fivegrams, coca_links, WORDS):
    noise_probabilities = [0.1, 0.2, 0.3, 0.4, 0.5]  
    # Сan be changed to have bigger test set
    max_words = 100
    for noise_probability in noise_probabilities:
        noisy_text = generate_test_set(text, noise_probability)
        words = noisy_text.split()[:max_words]
        noisy_text =' '.join(words)
        
        # Saving the noisy text to a file
        # with open(f"noisy_text_{noise_probability}.txt", "w") as f:
        #     f.write(noisy_text)
        
        # Apply the correction models
        my_model_text = neural_network_correction(noisy_text, bigrams, fivegrams, coca_links, WORDS)
        norvig_model_text = norvig_correction(noisy_text)
        my_accuracy = evaluate_correction(' '.join(text.split()[:max_words]), my_model_text)
        norvig_accuracy = evaluate_correction(' '.join(text.split()[:max_words]), norvig_model_text)

        print(f'Noise: {noise_probability}, My implementation accuracy: {my_accuracy}, Norvig implementation: {norvig_accuracy}')

def save_results(noise_probability, original_text, noisy_text, my_model_text, norvig_model_text, my_accuracy, norvig_accuracy):
    with open('results.txt', 'a') as f:
        f.write(f'Noise: {noise_probability}\n')
        f.write(f'Original text: {original_text}\n')
        f.write(f'Noisy text: {noisy_text}\n')
        f.write(f'My implementation: {my_model_text}\n')
        f.write(f'Norvig implementation: {norvig_model_text}\n')
        f.write(f'My implementation accuracy: {my_accuracy}\n')
        f.write(f'Norvig implementation accuracy: {norvig_accuracy}\n\n')

# Test correction models and save the results
def test_correction_models_with_saving(text, bigrams, fivegrams, coca_links, WORDS):
    noise_probabilities = [0.1, 0.2, 0.3, 0.4, 0.5]  
    # Сan be changed to have bigger test set
    max_words = 100
    for noise_probability in noise_probabilities:
        noisy_text = generate_test_set(text, noise_probability)
        words = noisy_text.split()[:max_words]
        noisy_text =' '.join(words)
        
        my_model_text = neural_network_correction(noisy_text, bigrams, fivegrams, coca_links, WORDS)
        norvig_model_text = norvig_correction(noisy_text)
        my_accuracy = evaluate_correction(' '.join(text.split()[:max_words]), my_model_text)
        norvig_accuracy = evaluate_correction(' '.join(text.split()[:max_words]), norvig_model_text)

        print(f'Noise: {noise_probability}, My implementation accuracy: {my_accuracy}, Norvig implementation: {norvig_accuracy}')
        save_results(noise_probability,' '.join(text.split()[:max_words]), noisy_text, my_model_text, norvig_model_text, my_accuracy, norvig_accuracy)

#Run the test
test_correction_models_with_saving(text, bigrams, fivegrams, coca_links, WORDS)

Noise: 0.1, My implementation accuracy: 0.93, Norvig implementation: 0.6
Noise: 0.2, My implementation accuracy: 0.84, Norvig implementation: 0.57
Noise: 0.3, My implementation accuracy: 0.71, Norvig implementation: 0.6
Noise: 0.4, My implementation accuracy: 0.53, Norvig implementation: 0.56
Noise: 0.5, My implementation accuracy: 0.53, Norvig implementation: 0.54


## Test correctors on the input text

In [8]:
def test_correction_models(text, bigrams, fivegrams, coca_links, WORDS):
    my_model_text = context_sensitive_correction(text, bigrams, fivegrams, coca_links, WORDS)
    norvig_model_text = norvig_correction(text)
    print("Input text:")
    print(text)
    print("\nText with spelling corrections (My implementation):")
    print(my_model_text)
    print("\nText with spelling corrections (Norvig's implementation):")
    print(norvig_model_text)

def check_correction_models(input_text, bigrams, fivegrams, coca_links, WORDS):
    my_model_text = context_sensitive_correction(input_text, bigrams, fivegrams, coca_links, WORDS)
    norvig_model_text = norvig_correction(input_text)
    print("Input text:") 
    print(input_text)
    print("\nText with spelling corrections (My implementation):")
    print(my_model_text)
    print("\nText with spelling corrections (Norvig's implementation):")
    print(norvig_model_text)
    

input_text = "1dking sport"
check_correction_models(input_text, bigrams, fivegrams, coca_links, WORDS)

Input text:
1dking sport

Text with spelling corrections (My implementation):
taking sport

Text with spelling corrections (Norvig's implementation):
taking sport
